In [1]:
# Load the "autoreload" extension so that code can change
%load_ext autoreload
# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
import gurobipy

from ctrace.constraint import ProbMinExposedMIP, ProbMinExposed
from ctrace.utils import PQ_deterministic, find_contours, find_excluded_contours
from ctrace.dataset import load_graph, load_sir, load_graph_cville
from ctrace.solve import basic_non_integer_round
import collections
import numpy as np

In [3]:
# Set seed
np.random.seed(42)

In [4]:
G = load_graph_cville()

In [5]:
N = len(G.nodes)
N

131219

In [6]:
p = 0.08

# Infected set
num_infected = int(N * p)
I = np.random.choice(N, num_infected, replace=False)
K = 500

In [7]:
# Contours
contour1, contour2 = find_excluded_contours(G, I, set())

# Conditionals
p1, q = PQ_deterministic(G, I, contour1, p)

In [8]:
# LP Solution
sol_lp = ProbMinExposed(G, I, contour1, contour2, p1, q, K)
sol_lp.solve_lp()

In [9]:
print(sol_lp.objectiveVal)
print(sol_lp.isOptimal)

15577.581257783917
True


In [10]:
# GUROBI MIP
sol = ProbMinExposedMIP(G, I, contour1, contour2, p1, q, K, costs=None, solver='GUROBI')
sol.solve_lp()

In [11]:
print(sol.objectiveVal)
sol_vec = sol.get_variables()
collections.Counter(sol_vec)

15587.819604629321


Counter({0.0: 79808, 1.0: 500})

In [12]:
sol.isOptimal


True

In [13]:
sol_lp = ProbMinExposed(G, I, contour1, contour2, p1, q, K)
rounding_sol = basic_non_integer_round(sol_lp)


In [14]:
rounding_sol[0]

15598.084578440454